<div style='display:flex;'>
    <div style='width:25%;text-align:right;'> <h4 style='color:#9A11DA'>Previous Module:</h4></div>
    <div style='width:25%;text-align:left;'> <h4 ><a href='./07_Multi_qubit_gates.ipynb' style='color:#9A11DA;text-decoration:underline;opacity:0.6;'>7. Multi-Qubit Gates</a></h4></div>
    <div style='width:25%;text-align:right;'> <h4 style='color:#9A11DA'>Next Module:</h4></div>
    <div style='width:25%;text-align:left;'> <h4 ><a href='./09_BB84_Protocol.ipynb' style='color:#9A11DA;text-decoration:underline;opacity:0.6;'>9. The BB84 Protocol</a></h4></div>
</div>
</div>
<!-- <hr style='border-color:grey;'> -->

In [1]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np

def U_f(circ_global, qubits):
    #the first argument is a circuit to which we want U_f to be applied,
    #the second argument is a list of qubit to which U_f is to be applied.
    
    ##### OR gate #####

    n=4
    m=n
    out_qubit = n+m-1

    # Build a sub-circuit
    circ = QuantumCircuit(2, name='U_f')

    circ.cx(0,1)   # a simple circuit implementing the function f such that f(0) = 1 and f(1) = 0.

    sub_inst = circ.to_instruction()

    q = circ_global.qubits
    circ_global.append(sub_inst, [q[qubits[i]] for i in range(len(qubits))])

<h1 style="color:#D30982;">Set-up of Deutsch's Algorithm</h1>

In 1985, David Deutsch proposed one of the very first quantum algorithms to provide evidence for the power of quantum computers over their classical counterparts. Specifically, this is one of the first examples of how quantum superposition and interference can be exploited to obtain computational advantages over classical algorithms. In this section, we'll learn the simple problem that Deutsch solved along with the quantum algorithm to solve it.

<h2 style="color:#9A11DA;"> Problem Statement </h2>

The goal of the problem is to determine whether a given function of a single bit $f: \{0,1\}$ &rarr; $\{0,1\}$ is *constant* or *balanced*. By constant, we mean that both inputs (0 and 1) to the function result in the same output, i.e. 

$$\begin{align} f(0)&=0 \quad  &\text{or} \quad \quad f(0)&=1 \\ f(1)&=0  \quad & \quad \quad f(1)&=1\end{align}$$ 

By *balanced*, we mean that the outputs are different for different inputs, i.e.

$$\begin{align} f(0)&=1 \quad  &\text{or} \quad \quad f(0)&=0 \\ f(1)&=0  \quad & \quad \quad f(1)&=1\end{align}$$

The figure below shows a visual representation of these four possibilities. The top two mappings represent constant functions. The bottom two mappings represent balanced functions.

<img src="images/constant_balanced_function_mappings.PNG" width=600 height=350 align="center" style='margin-left:auto;margin-right:auto;'/>

The problem of determing whether the function is constant or balanced is sometimes referred to as the Deutsch-Josza problem. In this problem, the function is given to you as a "black box," meaning that you get to *query* the value of the function for a particular input, and this is the only way you can access the function. 

<h3 style="color:#9A11DA;"> Classical Solution to the Deutsch-Josza Problem</h3>

Let's think about the problem from a classical point of view. How many queries to the "black box" does it take for us to be certain about whether $f$ is constant or balanced? We could start by inputting the bit value $0$ into the black box, and find that the black box returns some value $f(0)$. After doing this, we have no information about $f(1)$. If $f(0)=f(1)$, the function is constant, and if $f(0)\neq f(1)$, the function is balanced. Hence, knowing $f(1)$ is essential, and we can conclude that the black box requires two queries to get an answer to our problem.

The purpose of Deutsch's aglorithm is to do better. Since the classical solution involves two queries, we need to show that the quantum equivalent can solve this problem in just one query!

<h2 style="color:#9A11DA;"> Preliminaries </h2>

Before we outline Deutsch's algorithm, we describe a few mathematical preliminaries that will be useful.

<h3 style="color:#9A11DA;"> Parity of a Sum </h3>

We use the symbol $\oplus$ to denote the parity of the sum of two bits. Concretely:
- $a \oplus b = 0$ if $a+b$ is even, and
- $a \oplus b = 1$ if $a+b$ is odd.

<h3 style="color:#9A11DA;"> Querying a Function Quantumly</h3>

**Gate Implementation**. The first question that arises in the quantum setting is: how exactly do we query a black-box function using qubits? 

We should start by being more precise about what the 'black box' does. Notice first that we cannot model the black box as implementing the map $\left | x \right \rangle \rightarrow \left | f(x) \right \rangle$, i.e.

$$ \left | 0 \right \rangle \rightarrow \left | f(0) \right \rangle$$
$$ \left | 1 \right \rangle \rightarrow \left| f(1) \right \rangle$$

Why not? Consider the following mapping:

$$ \begin{align} \begin{array}{lcl}\left | 0 \right \rangle \rightarrow \left | 1 \right \rangle \\ \left | 1 \right \rangle \rightarrow \left| 0 \right \rangle \end{array} \end{align}$$

This mapping is unitary, and in fact is equivalent to the $X$ gate. However, not all of the mappings discussed above have this property. In general, the function $f(x)$ cannot be implemented directly as a unitary transformation, i.e. as a valid quantum gate. 

<font size=3 color=9A11DA>**Exercise.**</font> Write down a function $f$ for which the transformation given above is not a valid single qubit gate, i.e. it does not specify by a unitary linear transformation.

**A Workaround.** Instead, we will model the black box as acting as follows:

$$ \left | x \right \rangle \left | y \right \rangle \rightarrow \left | x \right \rangle \left | y \oplus f(x) \right \rangle$$

In particular, notice that $\left | x \right \rangle \left | 0 \right \rangle \rightarrow \left | x \right \rangle \left | f(x) \right \rangle$. One can check that the linear transformation implemented by the latter black box is unitary (and thus is a valid quantum gate). We denote this unitary transformation by $U_f$.

We will show that a quantum computer with access to $U_f$ can determine with certainty with a *single* query whether the function $f$ is balanced or not! 

><font size=3 color=9A11DA>*Remark.*</font> Notice that the problem of determining whether $f$ is constant or balanced is exactly equivalent to determining whether $f(0) \oplus f(1) = 0$ or $f(0) \oplus f(1) = 1$.

If you are worried about the new black box with the $\oplus$ operation being more powerful than the original classical box, you can convince yourself that a classical algorithm would require two queries even if it had access to the following black box:

$$ (x,y) \rightarrow (x, y \oplus f(x))$$

The reason is simple: each classical query gives information either about $f(0)$ or about $f(1)$, but crucially not both! And we need information about both to determine whether $f$ is constant or balanced.

<font size=3 color=9A11DA>**Exercise.**</font> Compute the action of $U_f$ on the standard basis elements.

<h3 style="color:#9A11DA;"> Quantum Superpositions</h3>

The goal of Deutsch's algorithm is to obtain the answer to our problem in a single query to the black-box function. Let's think about how we can use superpositions to make this happen. Recall that applying $U_f$ produces the following mapping:
$$ \left | x \right \rangle \left | y \right \rangle \rightarrow \left | x \right \rangle \left | y \oplus f(x) \right \rangle$$

If the state of the first qubit is $\left|0\right>$, we can obtain information about about $f(0)$. If the state of the first qubit is $\left|1\right>$, we can obtain information about about $f(1)$. Crucially, neither input state can give us both. 

**First Qubit in Superposition.** What about starting with the first qubit in a superposition state $\tfrac{1}{\sqrt{2}}(\left| 0 \right\rangle+\left| 1 \right\rangle)$? This means the overall input state is

$$ \left|\psi\right\rangle=\left(\tfrac{1}{\sqrt{2}}\left| 0 \right\rangle+\tfrac{1}{\sqrt{2}}\left| 1 \right\rangle\right)\left|0\right\rangle=\tfrac{1}{\sqrt{2}}\left| 0 \right \rangle \left| 0 \right\rangle +\tfrac{1}{\sqrt{2}}\left| 1 \right \rangle \left| 0 \right\rangle\,.$$

If we apply the unitary $U_f$ to this state (this counts as one query), we obtain 

$$\begin{align}
\label{eq: f0f1}\tag{1}
\left|\psi_{output}\right\rangle &=U_f\left|\psi\right\rangle \\ &= U_f\left(\tfrac{1}{\sqrt{2}}\left| 0 \right \rangle \left| 0 \right\rangle +\tfrac{1}{\sqrt{2}}\left| 1 \right \rangle \left| 0 \right\rangle \right) \\ &=\tfrac{1}{\sqrt{2}}\left| 0 \right \rangle \left| f(0)\right\rangle +\tfrac{1}{\sqrt{2}}\left| 1 \right \rangle \left| f(1) \right\rangle\,. 
\end{align}$$

where to obtain the last expression we used that, by definition, $U_f$ acts linearly. Now, we have queried the black box function only once, but we have obtained a state that contains both $f(0)$ and $f(1)$! We did this by placing the first qubit in a superposition state. However &mdash;and this very important &mdash; if one tries to 'extract' information about both $f(0)$ and $f(1)$ by measuring $\left|\psi_{output}\right\rangle$, then one would fail. A measurement of $\left|\psi_{output}\right\rangle$ would yield $f(0)$ with probability $1/2$ and $f(1)$ with probability $1/2$. Crucially, such a measurement would not reveal any information about $f(0)$ and $f(1)$ simultaneously. Let's summarize our results so far:
- By putting the *first* qubit in superposition, we can obtain a state that contains information about both $f(0)$ and $f(1)$.
- We cannot access both pieces of information simultaneously.

**Second Qubit in Superposition.** Let's try a different approach. Let's not put the first qubit in a superposition. Instead, let's place the *second* qubit in a superposition state! In particular, let's initialize the second qubit as the minus state $\left| - \right\rangle=\tfrac{1}{\sqrt{2}}(\left| 0 \right\rangle-\left| 1 \right\rangle)$. Let $\left|x\right\rangle$ be the state of the first qubit (i.e. either $\left|0\right\rangle$ or $\left|1\right\rangle$). In this case, our initial two-qubit state would be:

$$ \begin{align} \left|\psi\right\rangle &=\left|x\right\rangle\left|-\right\rangle   \\ &=\left|x\right\rangle\left(\tfrac{1}{\sqrt{2}}\left| 0 \right\rangle-\tfrac{1}{\sqrt{2}}\left| 1 \right\rangle\right)\\ &=\tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 0 \right\rangle -\tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 1 \right\rangle\,\end{align},$$

Applying $U_f$, we obtain

$$\begin{align}
\left|\psi_{output}\right\rangle &=U_f\left|\psi\right\rangle \\ &= U_f\left(\tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 0 \right\rangle -\tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 1 \right\rangle \right) \\ &= \tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 0 \oplus f(x) \right\rangle -\tfrac{1}{\sqrt{2}}\left| x \right \rangle \left| 1 \oplus f(x) \right\rangle \\ &= \tfrac{1}{\sqrt{2}}\left| x \right \rangle \Big( \left| 0 \oplus f(x) \right\rangle - \left| 1 \oplus f(x) \right\rangle \Big)
\end{align}$$

Our result here depends specificially on whether $f(x)=0$ or $f(x)=1$. In the two cases we have:

$$\begin{align} f(x)&=0\quad\quad & \left|\psi_{output}\right\rangle &= \tfrac{1}{\sqrt{2}}\left| x \right \rangle \big( \left| 0 \right\rangle - \left| 1 \right\rangle \big)= \left| x \right \rangle \left| - \right \rangle  \\ f(x)&=1 \quad\quad & \left|\psi_{output}\right\rangle &= \tfrac{1}{\sqrt{2}}\left| x \right \rangle \big( \left|1 \right\rangle - \left| 0 \right\rangle \big)=-\left| x \right \rangle\left| - \right \rangle   \end{align}$$

In these two cases, the only difference is an overall global phase of $-1$ in front. This means we can summarize our result as

<font color=b30c65> $$ U_f \left| x \right \rangle \left| - \right\rangle = (-1)^{f(x)} \left| x \right \rangle \left| - \right\rangle \,, \tag{2} \label{eq: crucial}$$  </font> 

This formulation of our answer is absolutely crucial. Now, instead of our answer being stored in the state itself, it is stored in the *phase* of the state. It turns out that storing the answer to our problem in this way will make it possible to extract information about both $f(0)$ and $f(1)$ simultaneously!

**Summary.**

- We can create a superposition over $f(0)$ and $f(1)$ from a single query to the black-box function $U_f$ by placing the *first* qubit in superposition, specifically the  $\left|+\right>$ state. However, this information cannot be simply extracted through measurement.
- We can encode the outcome $f(x)$ into the *phase* of a state by placing the *second* qubit in superposition, specificially the $\left|-\right>$ state.

Notice that while the second approach seems interesting, it does not yet allow us to obtain information about $f(0)$ and $f(1)$ simultaneously. In fact, the approach, as is, does not even reveal information about $f(x)$ for a single $x$. This is because the $x$-dependent $-1$ phase that appears is *global*. 

It turns out that there is a way to extract some information about both $f(0)$ and $f(1)$ simultaneously by combining the first and the second approach. This requires carefully orchestrating superpositions and interferences. In particular, it will require placing *both* qubits in superposition.

<font size=3 color=9A11DA>**Exercise.**</font> Show explicitly that $U_f \left| x\right>\left|-\right>=(-1)^{f(x)}\left| 0\right>\left|-\right>$.

<font size=3 color=9A11DA>**Exercise.**</font> In the expression above, is the phase $(-1)^{f(x)}$ a global phase or a relative phase? Explain your reasoning.

<font size=3 color=9A11DA>**Exercise.**</font> Consider the state

$$\left|\psi\right>=\tfrac{1}{\sqrt{2}}\Big((-1)^{f(0)}\left| 0\right>+(-1)^{f(1)}\left| 1\right>\Big).$$

Under what condition(s) on $f(0)$ and $f(1)$ does this state have a relative phase of $-1$? 

<h1 style="color:#D30982;"> Deutsch's Algorithm </h1>

Deutsch's algorithm provides a way to deterministically extract the bit $f(0) \oplus f(1)$, which, by the remark we made earlier, is precisely the answer to whether $f$ is constant or balanced: if $f(0) \oplus f(1) = 0$, then $f$ is constant, otherwise it is balanced. Deutsch's algorithm is truly quite magical. We understand that, until you internalize it, it might appear as if this algorithm has 'fallen from the sky'. In what follows, our goal is to provide as much intuition as possible to help you internalize it, and to have you realize that the idea is actually quite natural, especially after the observations made in the previous section.

The intuition is the following. Our goal is to build a circuit that acts on two qubits, initialized in the state $\left| 00\right \rangle$, overall makes a single query to $U_f$, and ends up outputting the two qubit state 
$$\left| f(0) \oplus f(1) \right \rangle \left| aux \right \rangle, \label{eq1} \tag{3}$$

where $\left| aux \right \rangle$ is some auxiliary state which we don't really care about in the end. Clearly, if we can achieve this, we have solved the problem, because we can just measure the first qubit, and the outcome of the measurement will be  '$f(0)\oplus f(1)$' with probability $1$.

<h3 style="color:#9A11DA;"> Constructive and Destructive Interference</h3>

Recall the section about *relative phase* in module 5 (if you have completely forgotten about it, we encourage you to look back), where we discussed the difference between the states $\left|+\right> =\frac{1}{\sqrt{2}}\left( \left| 0 \right> + \left| 1 \right> \right)$ and $\left|-\right> =\frac{1}{\sqrt{2}}\left( \left| 0 \right> - \left| 1 \right> \right)$. We determined that they differed by a relative phase angle $\pi$. Further, we realized that we could distinguish these two states by applying a Hadamard ($H$) gate, giving:

$$ H\left | + \right \rangle \rightarrow \left | 0 \right \rangle$$
$$ H\left | - \right \rangle \rightarrow \left| 1 \right \rangle$$

In the context of our algorithm, if we could somehow encode useful information in the relative phase, we could then measure that information indirectly by first applying a Hadamard gate. It's important to understand how this process happens. Consider the Hadamard on the $\left | + \right \rangle$ state:
$$\begin{align}
H \frac{\color{blue}{\left|0\right\rangle}+\color{red}{\left|1\right\rangle}}{\sqrt{2}} &= \frac{1}{\sqrt{2}} H \color{blue}{\left|0\right\rangle}+  \frac{1}{\sqrt{2}} H \color{red}{\left|1\right\rangle} \\
&= \frac{1}{\sqrt{2}} \left(\frac{\color{blue}{\left| 0 \right \rangle + \left| 1 \right \rangle}}{\sqrt{2}}\right) + \frac{1}{\sqrt{2}} \left(\frac{\color{red}{\left| 0 \right \rangle - \left| 1 \right \rangle}}{\sqrt{2}}\right) \\
&= \frac12 \left(\color{blue}{\left|0\right\rangle} +  \color{blue}{\left|1\right\rangle} +  \color{red}{\left|0\right\rangle} -  \color{red}{\left|1\right\rangle}\right) \\ 
&=\left| 0 \right \rangle
\end{align}$$
What happens here, is that the blue $\color{blue}{\left| 1 \right \rangle}$ and the red  $\color{red}{\left| 1 \right \rangle}$ have opposite phases, and *interfere destructively*. 

What we will try to accomplish in Deutsch's algorithm is to initialize the first qubit in superposition, and then modify its relative phase such that one possible answer to our problem corresponds to the $\left | + \right \rangle$ state, and the other corresponds to the $\left | - \right \rangle$ state.

Let's start by trying to reverse engineer a way to get to \eqref{eq1}. In particular, let's think about Quantum computing is about *amplitudes* and *interferences*, so let's think about how we could exploit these! We could try orchestrating a computation in which amplitudes corresponding to the bit $f(0) \oplus f(1)$ interfere *constructively*, while amplitudes corresponding to the opposite bit (denoted as $\overline{f(0) \oplus f(1)}$) to interfere *destructively*. Let's be a bit more concrete. Consider the following expression:

$$ \frac{1}{2} \big(\left| 0 \right \rangle + \left| 1 \right \rangle + (-1)^{f(0)\oplus f(1)} (\left| 0 \right \rangle - \left| 1 \right \rangle) \big) \label{eq3} \tag{3}$$

You can check for yourself that this expression, upon simplification, gives precisely $\left| f(0) \oplus f(1) \right \rangle$. The key is the phase of $(-1)^{f(0)\oplus f(1)}$, which makes things interfere in precisely the correct way depending on the value of $f(0) \oplus f(1)$.

<font size=3 color=9A11DA>**Exercise.**</font> Simplify expression (3) above for the two cases:
- $f(0) \oplus f(1)=0$
- $f(0) \oplus f(1)=1$

In both cases, identify which components of the superposition interfere *contructively* and which components interfere *destructively*.

<h3 style="color:#9A11DA;"> Implementation</h3>

Motivated by the discussion so far, we prepare the first qubit in the equal superposition $\left|+ \right\rangle = \left(\color{blue}{\left|0\right\rangle}+\color{red}{\left|1\right\rangle}\right)/\sqrt{2}$, and the second qubit in the state $\left| - \right\rangle$. We can prepare such a state by starting with $\left| 0\right\rangle \left| 1\right \rangle$ and applying $HH$:

$$H H \left(\left|0\right\rangle \left|1\right\rangle \right) =\frac{\color{blue}{\left|0\right\rangle}+\color{red}{\left|1\right\rangle}}{\sqrt{2}}\left| - \right\rangle $$ 

Next, we apply $U_f$:

$$ \begin{align}  U_f &\left( \frac{\color{blue}{\left|0\right\rangle}+\color{red}{\left|1\right\rangle}}{\sqrt{2}}\left| - \right\rangle \right) \\ U_f &\left( \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right \rangle} \left| - \right\rangle \right)+ U_f\left( \frac{1}{\sqrt{2}}\color{red}{\left| 1 \right \rangle} \left| - \right\rangle \right)\\ & \left( \color{blue}{(-1)^{f(0)}} \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right \rangle} \left| -\right\rangle \right) +  \left( \color{red}{(-1)^{f(1)}}\frac{1}{\sqrt{2}}\color{red}{\left| 1 \right \rangle} \left| - \right\rangle \right) \,,
\end{align}$$

where we used the crucial observation of Equation (3). We can rewrite the resulting expression by pulling out a factor of$\color{blue}{(-1)^{f(0)}}$, as follows.

$$\begin{align} & \left( \color{blue}{(-1)^{f(0)}} \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right \rangle} \left| -\right\rangle \right)  +  \left( \color{blue}{(-1)^{f(0)}} \color{blue}{(-1)^{f(0)}} \color{red}{(-1)^{f(1)}}\frac{1}{\sqrt{2}}\color{red}{\left| 1 \right \rangle} \left| - \right\rangle \right) \\  =\, &\color{blue}{(-1)^{f(0)}} \left( \left( \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right \rangle} \left| -\right\rangle \right) +  \left( \color{blue}{(-1)^{f(0)}} \color{red}{(-1)^{f(1)}}\frac{1}{\sqrt{2}}\color{red}{\left| 1 \right \rangle} \left| - \right\rangle \right) \right) \\ =\, & \color{blue}{(-1)^{f(0)}}\left( \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right\rangle} + (-1)^{\color{blue}{f(0)}+\color{red}{f(1)}}\frac{1}{\sqrt{2}} \color{red}{\left| 1 \right \rangle} \right) \left| - \right\rangle \end{align} $$
where we used basic properties of exponentiation to collect exponents in the second term. We also made use of the fact that $\color{blue}{(-1)^{f(0)}}\color{blue}{(-1)^{f(0)}}=1$ for either possible value of $f(0)$.

Notice that the global phase of $\color{blue}{(-1)^{f(0)}}$ will not affect probabilities of measurement outcomes (since they only depend on magnitudes of the amplitudes), so we can disregard it. This leaves us with the following state:

$$\left( \frac{1}{\sqrt{2}}\color{blue}{\left| 0 \right\rangle} + (-1)^{\color{blue}{f(0)}+\color{red}{f(1)}}\frac{1}{\sqrt{2}} \color{red}{\left| 1 \right \rangle} \right) \left| - \right\rangle .$$

At this point, the second qubit has served its purpose, and we can ignore it from now on. The key observation is the following:

- If $f(0)\oplus f(1) = 0$, the relative phase is $(-1)^0 = 1$, and the final state simplifies to $\left|+ \right\rangle$.
- If $f(0)\oplus f(1) = 1$, the relative phase is $(-1)^1 = -1$, and the final state simplifies to $\left|- \right\rangle$.

Thus, we have achieved our goal: if $f$ is constant, the state will be $\left|+ \right\rangle$, and if $f$ is balanced, the state will be $\left|- \right\rangle$. As we have discussed previously, applying a Hadamard, followed by a measurement distinguishes these two cases perfectly!

<h3 style="color:#9A11DA;">Recap of Deutsch's Algorithm</h3>

Let's recap the algorithm we just described. After all is said and done, the algorithm is pretty simple:

- Prepare the state $\left| + \right\rangle \left| - \right\rangle$;
- Invoke an application of $U_f$;
- Undo $H$ on the first qubit (i.e. apply $H$ again, since $H$ is its own inverse.);
- Measure the first qubit.

<h1 style="color:#D30982;">Apply Deutsch's Algorithm</h1>

Let $f: \{0,1\} \rightarrow \{0,1\}$ be some unknown function from one bit to one bit. In the next exercise, you are given a unitary $U_f$ as a black-box, meaning that you get to "invoke" it and use it, but you don't get to see how it is built. You are promised that $U_f$ acts on two qubits as follows:

$$ (x,y) \rightarrow (x, y \oplus f(x))$$

Your task is to determine, making just a single invocation to $U_f$, whether $f$ is constant (i.e. $f(0) = f(1)$) or balanced (i.e. $f(0) \neq f(1)$).

In the next cell, we show you how to invoke $U_f$.

In [ ]:
from qiskit import *
from python_functions.U_f import U_f

circ = QuantumCircuit(2)

U_f(circ, [0,1])

circ.draw()

<font size=3 color=9A11DA>**Exercise.**</font> Using just a single invocation of $U_f$, determine whether the function $f$ is constant or balanced.

<font color=9A11DA>**Solution.**</font> In the following cell we draw the circuit implementing Deutsch's algorithm. 

In [ ]:
from qiskit import *
from python_functions.U_f import U_f

circ = QuantumCircuit(2, 1) 

The first argument means that we prepare a quantum circuit with two qubits. The second argument means that we'll measure only on qubit, and store one bit as outcome. Next we initialize the state $\left|+\right>$ on the first qubit and the state $\left|-\right>$ on the second qubit. Recall that indices start counting from 0, so 1 represents the second qubit.

In [ ]:
circ.x(1) #x on second qubit
circ.h(1) #hadamard on second qubit
circ.h(0); #hadamard on first qubit

This creates the combined state $\left|+\right>\left|-\right>$. Now we can run the rest of the algorithm.

In [ ]:
U_f(circ, [0,1]) # invoke U_f

circ.h(0) # undo H on the first qubit

circ.draw() #draw the circuit

Next, we measure the first qubit. The first argument indicates that we measure the first qubit; the second argument that we store the outcome in first output bit declared at the start. After this, we can run execute the algorithm the same way we have in many examples so far.

In [ ]:
circ.measure(0,0) #measure

# Import Aer and execute
from qiskit import Aer, execute

backend_sim = Aer.get_backend('qasm_simulator')
sim = execute(circ, backend_sim, shots=1000) #how many times the quantum circuit is run.

sim_result = sim.result() # The results from the execution are stored in 'sim_result' and can be obtained using
counts = sim_result.get_counts(circ) #get counts
print(counts)

#plot histogram
from qiskit.visualization import plot_histogram
plot_histogram(counts)

The outcome of the measurement is $1$ all the time. We deduce that the function $f$ is balanced.

<h1 style="color:#D30982;">Review</h1>

- Deutsch's algorithm is one of the first examples of quantum computers exhibiting advantage over classical algorithms. We showed that Deutsch's algorithm can determine whether a function is *constant* or *balanced* using just one query to a black-box implementing the function.
- *Superposition* is often used to run a function on different inputs simultaneously: by placing the input qubit in the superposition state $\left|+\right>$, we are able to obtain a superposition over the outcomes $f(0)$ and $f(1)$. However, this doesn't allows us to directly obtain information about $f(0)$ and $f(1)$ simultaneously.
- By storing information about $f(0)\oplus f(1)$ in the *relative phase* between the two components of the initial superposition, we were able to exploit *constructive* and *destructive* interference to extract $f(0)\oplus f(1)$, i.e. the answer to whether the function is balanced or not.

<h1 style="color:#D30982;">Exercises</h1>

<h2 style="color:#9A11DA;">Multiple Choice</h2> 

1. What does it mean to have a "black-box" function?
    - you don't know exactly how it works, but you can find out by testing it on various inputs
    - the black-box function itself changes depending on what input you give it
    - there are an infinite number of possible outcomes of a black-box function
    - all of the above

2. What classical logic gate represents the parity operator $\oplus$?
    - AND
    - NAND
    - OR
    - XOR

3. Which concepts play a crucial role in Deutsch's Algorithm?
    - constructive and destructive interference
    - relative phase
    - global phase
    - the lossy nature of quantum measurement
    - all of the above

4. True or False: Deutsch's algorithm is *probabilistic*. We need to apply the algorithm a large number of times to be sure of the outcome.

<h2 style="color:#9A11DA;">Free Response</h2> 

1. In the set-up for Detusch's algorithm, why do we convert the black-box function $f(x)$ to a different function $U_f$?

2. We saw that placing the first qubit in the $\newcommand{\ket}[1]{\left| #1 \right\rangle}$$\ket{+}$ state allowed us to obtain a state that included $\ket{0}$ and $\ket{1}$. What prevented us from developing a solution to our problem?

3. What is significant about placing the second qubit in the $\ket{-}$ state before applying $U_f$? Would this benefit have worked if we had used placed the second qubit in the $\ket{+}$ state?

4. Review the summary section at the end our description of Deutsch's algorithm. With the exception of $U_f$, the steps only involve the first qubit. What role does the second qubit play in this algorithm? What kept us from implementing this algorithm using only one qubit?

<h2 style="color:#9A11DA;">Challenge</h2> 

1. Suppose we were allowed to invoke a different black-box $U_f$ behaving as follows:

$$ U_f \left| x \right \rangle \left| - \right\rangle = (-i)^{f(x)} \left| x \right \rangle \left| - \right\rangle $$

1. Apply $U_f$ to the state $\left|+\right>\left|-\right>$.
2. Unlike in the analysis of Deutsch's algorithm, factoring out a global phase of $(-i)^{f(0)}$ is not possible here. Why?
3. There are four possibilities for how the function $f$ behaves. Simplify the state in each of the four cases, keeping the *balanced* and *constant* cases grouped together. Simplify the states by removing global phases, such that the coefficient of $\left|0\right>$ is real and positive in all four cases. What is the relative phase of the *constant* states? What is the relative phase angle $\phi$ of the *balanced* states?
4. Now apply the Hadamard gate to your simplified states as an attempt to obtain the state $\left|f(0)\oplus f(1)\right>$ in the first qubit? Did the states interfere *destructively*? Why or why not? Explain your answer in terms of the phase angle $\phi$.

<hr>
<div style='display:flex;'>
    <div style='width:25%;text-align:right;'> <h4 style='color:#9A11DA'>Previous Module:</h4></div>
    <div style='width:25%;text-align:left;'> <h4 ><a href='./07_Multi_qubit_gates.ipynb' style='color:#9A11DA;text-decoration:underline;opacity:0.6;'>7. Multi-Qubit Gates</a></h4></div>
    <div style='width:25%;text-align:right;'> <h4 style='color:#9A11DA'>Next Module:</h4></div>
    <div style='width:25%;text-align:left;'> <h4 ><a href='./09_BB84_Protocol.ipynb' style='color:#9A11DA;text-decoration:underline;opacity:0.6;'>9. The BB84 Protocol</a></h4></div>
</div>
<!-- <hr style='border-color:grey;'> -->